# Taking advantage of Colab Pro



## Faster GPUs

With Colab Pro you have priority access to our fastest GPUs. For example, you may get a T4 or P100 GPU at times when most users of standard Colab receive a slower K80 GPU. You can see what GPU you've been assigned at any time by executing the following cell.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun  6 17:58:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator dropdown to GPU.

## More memory

With Colab Pro you have the option to access high-memory VMs when they are available. To set your notebook preference to use a high-memory runtime, select the Runtime > 'Change runtime type' menu, and then select High-RAM in the Runtime shape dropdown.

You can see how much memory you have available at any time by running the following code.


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). While Colab Pro subscribers still have limits, these will be roughly twice the limits for non-subscribers.

## Resource limits in Colab Pro

Your resources are not unlimited in Colab Pro. To make the most of Colab Pro, please avoid using resources when you don't need them. For example, only use a GPU or high-RAM runtime when required, and close Colab tabs when finished.


## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro and would be interested in a product with higher usage limits, do let us know.

If you encounter errors or other issues with billing (payments) for Colab Pro, please email colab-billing@google.com.

In [5]:
import os
import random
import numpy as np
import pandas as pd 
from skimage import io
from skimage import color
from PIL import Image
import PIL.Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from dask.array.image import imread
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras import optimizers


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [6]:
import pickle
train_image = []
image_label = []

for i in range(10):
  #path = "/content/drive/My Drive/kaggle/cache/train_r_224_c_224_c_3_class" + str(i) + ".dat"
  #path = "/content/drive/My Drive/distdrv/cache/mask_r_224_c_224_c_3_class" + str(i) + ".dat"
  path = "/content/drive/My Drive/distdrv/cache/zoommask_r_224_c_224_c_3_class" + str(i) + ".dat"

  
  print(f'loading pickle files from class = {i}')
  # get orig image
  file = open(path, 'rb')
  images, labels = pickle.load(file)
  train_image = train_image + images
  #image_label = image_label + labels





loading pickle files from class = 0
loading pickle files from class = 1
loading pickle files from class = 2
loading pickle files from class = 3
loading pickle files from class = 4
loading pickle files from class = 5
loading pickle files from class = 6
loading pickle files from class = 7
loading pickle files from class = 8
loading pickle files from class = 9


In [0]:
images = []
labels = []
driver_details = []

In [8]:
print(f'train image size = {len(train_image)}')

train image size = 22424


In [0]:
import random

random.shuffle(train_image)



In [10]:
print(f'train image size = {len(train_image)}')

train image size = 22424


In [0]:
## getting list of driver names

D = []
for features,labels,drivers in train_image:
    D.append(drivers)

## Deduplicating drivers

deduped = []

for i in D:
    if i not in deduped:
        deduped.append(i)
    

## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 4)
for i in driv_nums:
    driv_selected.append(deduped[i])


In [12]:
driv_nums

[17, 22, 14, 21]

In [13]:
len(deduped)

26

In [14]:
## Splitting the train and test

X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

#for features,labels,drivers,features2,labels2,drivers2 in zip(train_image, train_image2):
for features, labels, drivers in train_image:

    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        #D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        #D_train.append(drivers)


true_test = y_test
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))






19357 3067
19357 3067


In [15]:
len(X_train)

19357

In [16]:
## Converting images to nparray. Encoding the Y

X_train = np.array(X_train).reshape(-1,224,224,3)
X_test = np.array(X_test).reshape(-1,224,224,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)


(19357, 224, 224, 3)
(3067, 224, 224, 3)
(19357, 10)
(3067, 10)


In [0]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
  #print(var, sys.getsizeof(obj))
  pass

In [18]:
## Defining the input
from keras.layers import Input
xception_input = Input(shape = (224, 224, 3), name = 'Image_input')

## The RESNET model

from keras.applications.xception import preprocess_input, decode_predictions
from keras.applications.xception import Xception


#Get the RESNET weights and layers

model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
model_xception_conv.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [19]:
#Use the generated model 
from keras.models import Model


output_xception_conv = model_xception_conv(xception_input)

#Add the fully-connected layers 

x=GlobalAveragePooling2D()(output_xception_conv)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # **reduce dropout 
x=Dense(1024,activation='relu')(x) #dense layer 2
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(512,activation='relu')(x) #dense layer 3
x = Dense(10, activation='softmax', name='predictions')(x)


xception_pretrained = Model(input = xception_input, output = x)
# for layer in resnet50_pretrained.layers[:2]:
#     layer.trainable=False
# for layer in resnet50_pretrained.layers[2:]:
#     layer.trainable=True


xception_pretrained.summary()


# Compile CNN model
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
sgd = optimizers.SGD(lr = 0.005)
xception_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_5 (Batch (None, 1024)              4096

In [20]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer = ModelCheckpoint('/content/drive/My Drive/kaggle/xceptionmaskModel_aug.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)


datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
xception_model = xception_pretrained.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[checkpointer, earlystopper],
                                                            epochs = 30, verbose = 1, validation_data = (X_test, y_test))


Epoch 1/30
303/302 [==============================] - 344s 1s/step - loss: 2.3027 - accuracy: 0.1741 - val_loss: 2.2231 - val_accuracy: 0.2938

Epoch 00001: val_loss improved from inf to 2.22307, saving model to /content/drive/My Drive/kaggle/xceptionmaskModel_aug.hdf5
Epoch 2/30
303/302 [==============================] - 327s 1s/step - loss: 1.8078 - accuracy: 0.3636 - val_loss: 1.3755 - val_accuracy: 0.5634

Epoch 00002: val_loss improved from 2.22307 to 1.37551, saving model to /content/drive/My Drive/kaggle/xceptionmaskModel_aug.hdf5
Epoch 3/30
303/302 [==============================] - 327s 1s/step - loss: 1.3043 - accuracy: 0.5464 - val_loss: 0.8363 - val_accuracy: 0.7069

Epoch 00003: val_loss improved from 1.37551 to 0.83634, saving model to /content/drive/My Drive/kaggle/xceptionmaskModel_aug.hdf5
Epoch 4/30
303/302 [==============================] - 326s 1s/step - loss: 1.0269 - accuracy: 0.6453 - val_loss: 0.7017 - val_accuracy: 0.7551

Epoch 00004: val_loss improved from 0.

In [0]:
#vgg16_model.save("/content/drive/My Drive/kaggle/singleModel_aug.h5")
xception_pretrained.save_weights("/content/drive/My Drive/kaggle/xceptionmaskModel_aug_weights.h5")

In [28]:
from keras.models import load_model
rcModel = load_model('/content/drive/My Drive/kaggle/xceptionmaskModel_aug.hdf5')
rcModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_5 (Batch (None, 1024)              4096

In [29]:
X_test.shape

(3067, 224, 224, 3)

In [30]:
# labels is the image array

from sklearn.metrics import accuracy_score, confusion_matrix

model1_prediction = []
model1_pred_class = []

model1_prediction = rcModel.predict(X_test)
print('Images Predicted until now:',len(model1_prediction))
print(f'True images: {len(true_test)}')

for i in range(len(model1_prediction)):
    model1_pred_class.append(np.where(model1_prediction[i] == np.amax(model1_prediction[i]))[0][0])

    
print('The accuracy of this model over validation set is:',accuracy_score(true_test,model1_pred_class))
confusion_matrix(true_test,model1_pred_class)

Images Predicted until now: 3067
True images: 3067
The accuracy of this model over validation set is: 0.8474078904466906


array([[205,   1,   0,   0,   1,  21,   1,   0,  17,  43],
       [  7, 267,   0,   0,   0,   0,   4,   2,   1,   7],
       [  0,   0, 268,   0,   0,   0,   6,   0,  60,   0],
       [  1,   2,   0, 292,   4,   1,   1,   0,  11,   1],
       [  1,   0,   1,   3, 294,   0,   1,   0,  17,   1],
       [  3,   0,   0,   0,   0, 315,   0,   0,   0,   0],
       [  0,  21,   0,   0,   0,   0, 298,   0,   4,   0],
       [  0,   0,   0,   0,   0,   1,   0, 277,   1,   0],
       [  7,   0,   4,   0,   1,   1,  67,   5, 200,   2],
       [ 68,   1,   0,   1,   1,  26,   0,  34,   4, 183]])

In [25]:
print(f'prediction={model1_prediction}, shape={model1_prediction.shape}')

prediction=[[7.8515772e-04 9.9393433e-01 4.3523800e-05 ... 3.6568981e-05
  4.9636350e-04 2.0862906e-03]
 [1.3213544e-04 2.7401693e-04 4.6475622e-04 ... 2.3031757e-04
  2.2209704e-02 3.3677058e-04]
 [4.2811334e-06 4.1560465e-06 9.8986560e-01 ... 2.9833677e-06
  9.4424039e-03 1.9848729e-05]
 ...
 [8.9626810e-05 3.2739728e-04 6.2108855e-04 ... 3.9202248e-04
  2.0481825e-03 7.7055156e-04]
 [2.0210673e-04 1.2613500e-03 2.5991503e-05 ... 1.1297927e-03
  2.4344025e-02 2.9230684e-03]
 [4.3133627e-05 2.3499433e-05 4.2250651e-01 ... 5.7906494e-04
  5.2107799e-01 4.2932166e-04]], shape=(3067, 10)


In [32]:
from sklearn.metrics import log_loss

lgloss = log_loss(y_test, model1_prediction, eps=1e-15, normalize=True, sample_weight=None, labels=None)

print('The log loss from this model is:',round(lgloss,2))

The log loss from this model is: 0.45
